---
title: "Advanced Statistical Process Control"
format: html
---

# 🏭 Advanced Process Monitoring & Control
## Portfolio Project 6 — Multivariate SPC, PCA-Based Hotelling T², EWMA + CUSUM Fusion, Fault Isolation & Root-Cause Diagnosis

---

### What This Notebook Covers (Beyond Basics)
| Topic | Technique |
|---|---|
| Multivariate SPC | Hotelling T² statistic (PCA-reduced) |
| Signal decomposition | PCA contribution analysis for fault isolation |
| Adaptive monitoring | EWMA + CUSUM joint detection with auto-tuned parameters |
| Fault isolation | Structured residual analysis (SRA) |
| Root-cause ranking | Contribution plot — identifies which variable caused the alarm |
| Phase I / Phase II | Proper two-phase control chart methodology |

### Dataset  
**Tennessee Eastman Process** (synthetic replica, 6 process variables)  
Reference: https://github.com/Ramin-Khalatbari/dataset-Tennessee-Eastman

---


In [ ]:
# ─── 1. Imports ─────────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
print('✓ All imports loaded.')

In [ ]:
# ─── 2. Synthetic Tennessee-Eastman style process ──────────
def gen_te_process(n=3000, seed=55):
    rng = np.random.default_rng(seed)

    # 6 correlated process variables (normal state)
    # Covariance structure: block-diagonal with cross-links
    cov = np.array([
        [1.00, 0.60, 0.20, 0.10, 0.05, 0.00],
        [0.60, 1.00, 0.35, 0.15, 0.08, 0.02],
        [0.20, 0.35, 1.00, 0.40, 0.10, 0.05],
        [0.10, 0.15, 0.40, 1.00, 0.45, 0.12],
        [0.05, 0.08, 0.10, 0.45, 1.00, 0.50],
        [0.00, 0.02, 0.05, 0.12, 0.50, 1.00],
    ])
    means = np.array([85.0, 3.2, 50.0, 0.45, 220.0, 68.0])
    stds = np.array([1.5, 0.2, 3.0, 0.03, 8.0, 2.0])

    # Generate correlated normal data
    Z = rng.multivariate_normal(np.zeros(6), cov, n)
    data = means + Z * stds

    labels = np.zeros(n, dtype=int)  # 0 = normal

    # ─── Fault 1: Gradual drift in Var 0 (Temperature) ───
    drift_start = 500
    drift_end = 700
    drift_amp = 3.0 * stds[0]
    ramp = np.linspace(0, drift_amp, drift_end - drift_start)
    data[drift_start:drift_end, 0] += ramp
    labels[drift_start:drift_end] = 1

    # ─── Fault 2: Step change in Var 2 (Flow) + correlated effect on Var 3 ───
    step_start = 1100
    step_end = 1300
    data[step_start:step_end, 2] += 2.5 * stds[2]
    data[step_start:step_end, 3] -= 0.8 * stds[3]   # downstream effect
    labels[step_start:step_end] = 2

    # ─── Fault 3: Variance increase in Var 4 + Var 5 (sensor noise) ───
    var_start = 1800
    var_end = 2000
    data[var_start:var_end,
         4] += rng.normal(0, 2.5 * stds[4], var_end - var_start)
    data[var_start:var_end,
         5] += rng.normal(0, 2.0 * stds[5], var_end - var_start)
    labels[var_start:var_end] = 3

    # ─── Fault 4: Correlation structure breakdown ───
    corr_start = 2300
    corr_end = 2500
    # Add independent noise to break the correlation
    data[corr_start:corr_end,
         :] += rng.normal(0, 0.5, (corr_end - corr_start, 6)) * stds
    labels[corr_start:corr_end] = 4

    var_names = ['Temperature', 'Pressure',
                 'FlowRate', 'Conc_A', 'HeatDuty', 'Humidity']
    df = pd.DataFrame(data, columns=var_names)
    df['Fault'] = labels
    df['Time'] = pd.date_range('2024-01-01', periods=n, freq='5min')
    return df, var_names


df, VAR_NAMES = gen_te_process()
print(f'Shape: {df.shape}')
print(
    f'Fault distribution: {dict(zip(*np.unique(df["Fault"], return_counts=True)))}')

1. Phase I — Establish Baseline (Normal Operating Region)

We use the first 500 samples (all normal) to compute the **in-control** mean, covariance, and PCA model. All subsequent monitoring uses these as reference.


In [ ]:
# ─── 3. Phase I: baseline statistics ─────────────────────
from scipy.stats import f as f_dist
PHASE1_END = 500   # all normal

phase1 = df[VAR_NAMES].iloc[:PHASE1_END].values
scaler = StandardScaler()
phase1_s = scaler.fit_transform(phase1)

# PCA on Phase I data
N_PC = 4   # retain 4 principal components
pca = PCA(n_components=N_PC)
pca.fit(phase1_s)

print(f'Phase I samples: {PHASE1_END}')
print(
    f'PCA explained variance: {pca.explained_variance_ratio_.sum()*100:.1f}% ({N_PC} PCs)')
print(f'Per-PC: {[f"{v*100:.1f}%" for v in pca.explained_variance_ratio_]}')

# Compute Phase I T² distribution for threshold calibration
scores_phase1 = pca.transform(phase1_s)
# T² = score @ diag(1/eigenvalues) @ score.T  (per sample)
eigvals = pca.explained_variance_
T2_phase1 = np.sum(scores_phase1**2 / eigvals, axis=1)

# UCL: use F-distribution approximation
p, n1 = N_PC, PHASE1_END
alpha = 0.01   # significance level
F_crit = f_dist.ppf(1 - alpha, p, n1 - p)
T2_UCL = p * (n1 - 1) * (n1 + 1) / (n1 * (n1 - p)) * F_crit

print(f'\nHotelling T² UCL (α={alpha}): {T2_UCL:.2f}')
print(f'Phase I T² max: {T2_phase1.max():.2f}  (should be < UCL)')

2. Phase II — Hotelling T² Monitoring


In [ ]:
# ─── 4. Phase II: compute T² for all samples ─────────────
X_all_s = scaler.transform(df[VAR_NAMES].values)
scores_all = pca.transform(X_all_s)
T2_all = np.sum(scores_all**2 / eigvals, axis=1)

# Also compute SPE (Squared Prediction Error) = reconstruction error
recon_all = pca.inverse_transform(scores_all)
SPE_all = np.sum((X_all_s - recon_all)**2, axis=1)

# SPE UCL (chi-squared approximation)
SPE_phase1 = np.sum(
    (phase1_s - pca.inverse_transform(pca.transform(phase1_s)))**2, axis=1)
n_resid = len(VAR_NAMES) - N_PC
SPE_UCL = np.percentile(SPE_phase1, 99)   # empirical 99th percentile

print(f'SPE UCL (empirical 99%): {SPE_UCL:.4f}')

# ─── Plot T² and SPE control charts ───
fig, axes = plt.subplots(2, 1, figsize=(16, 8), sharex=True)

# T²
axes[0].plot(T2_all, lw=0.7, color='steelblue')
axes[0].axhline(T2_UCL, color='red', ls='--',
                lw=1.2, label=f'UCL={T2_UCL:.1f}')
violations_T2 = T2_all > T2_UCL
axes[0].scatter(np.where(violations_T2)[0], T2_all[violations_T2],
                s=15, color='red', zorder=5, label=f'Violations ({violations_T2.sum()})')
# Shade fault regions
fault_colors = {1: '#ff9999', 2: '#ffcc99', 3: '#cc99ff', 4: '#99ccff'}
for fid, fc in fault_colors.items():
    mask = df['Fault'].values == fid
    if mask.any():
        idxs = np.where(mask)[0]
        axes[0].axvspan(idxs[0], idxs[-1], alpha=0.12,
                        color=fc, label=f'Fault {fid}')
axes[0].set_title('Hotelling T² Control Chart (Phase II)', fontsize=13)
axes[0].set_ylabel('T²')
axes[0].legend(loc='upper right', fontsize=7)

# SPE
axes[1].plot(SPE_all, lw=0.7, color='purple')
axes[1].axhline(SPE_UCL, color='red', ls='--',
                lw=1.2, label=f'UCL={SPE_UCL:.3f}')
violations_SPE = SPE_all > SPE_UCL
axes[1].scatter(np.where(violations_SPE)[0], SPE_all[violations_SPE],
                s=15, color='red', zorder=5, label=f'Violations ({violations_SPE.sum()})')
for fid, fc in fault_colors.items():
    mask = df['Fault'].values == fid
    if mask.any():
        idxs = np.where(mask)[0]
        axes[1].axvspan(idxs[0], idxs[-1], alpha=0.12, color=fc)
axes[1].set_title('SPE (Squared Prediction Error) Control Chart', fontsize=13)
axes[1].set_ylabel('SPE')
axes[1].set_xlabel('Sample')
axes[1].legend(loc='upper right', fontsize=7)

plt.tight_layout()
plt.show()

3. Fault Isolation — PCA Contribution Analysis


In [ ]:
# ─── 5. Contribution plot for a specific alarm ─────────
def contribution_plot(sample_idx, X_s, pca, eigvals, T2_UCL):
    """
    Compute the contribution of each original variable to the T² statistic.
    Uses the PCA loading decomposition method.
    """
    x = X_s[sample_idx]
    scores = pca.transform(x.reshape(1, -1))[0]

    # T² decomposition: sum over PCs of (score_k^2 / eigenval_k)
    # Each PC's contribution to each variable via loadings
    loadings = pca.components_   # (n_pc, n_vars)
    contributions = np.zeros(len(x))

    for k in range(len(eigvals)):
        pc_contrib = (scores[k]**2) / eigvals[k]
        # Distribute this PC's contribution to variables proportionally to |loading|²
        loading_sq = loadings[k]**2
        contributions += pc_contrib * loading_sq / (loading_sq.sum() + 1e-10)

    return contributions


# Find the first T² violation in each fault
print('Contribution analysis for first alarm in each fault:')
fig, axes = plt.subplots(2, 2, figsize=(14, 9))
fault_ids = [1, 2, 3, 4]

for ax, fid in zip(axes.flatten(), fault_ids):
    # First violation in this fault window
    mask = (df['Fault'].values == fid) & violations_T2
    if not mask.any():
        # Use the first sample of the fault even if no violation
        mask = df['Fault'].values == fid
    first_alarm = np.where(mask)[0][0]

    contribs = contribution_plot(first_alarm, X_all_s, pca, eigvals, T2_UCL)

    colors = ['#c44e52' if c == contribs.max() else '#4c72b0' for c in contribs]
    ax.bar(VAR_NAMES, contribs, color=colors, edgecolor='white')
    ax.set_title(
        f'Fault {fid} — Contribution Plot (sample {first_alarm})', fontsize=11)
    ax.set_ylabel('Contribution to T²')
    ax.tick_params(axis='x', rotation=20)
    # Annotate the dominant variable
    dom_var = VAR_NAMES[np.argmax(contribs)]
    ax.text(0.5, 0.92, f'↑ {dom_var}', transform=ax.transAxes,
            ha='center', fontsize=10, color='crimson', fontweight='bold')

plt.suptitle('Fault Isolation — PCA Contribution Plots', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

4. EWMA + CUSUM Joint Detection with Auto-Tuned Parameters


In [ ]:
# ─── 6. Joint EWMA-CUSUM on the T² statistic ─────────────
# EWMA on T²
lambda_ewma = 0.15
mu_T2 = T2_phase1.mean()
sigma_T2 = T2_phase1.std()

ewma_T2 = np.zeros(len(T2_all))
ewma_T2[0] = T2_all[0]
ewma_UCL = np.zeros(len(T2_all))
L_ewma = 2.5

for i in range(1, len(T2_all)):
    ewma_T2[i] = lambda_ewma * T2_all[i] + (1 - lambda_ewma) * ewma_T2[i-1]
    var_e = (sigma_T2**2 * lambda_ewma / (2 - lambda_ewma)) * \
        (1 - (1-lambda_ewma)**(2*(i+1)))
    ewma_UCL[i] = mu_T2 + L_ewma * np.sqrt(var_e)

ewma_alerts = ewma_T2 > ewma_UCL

# CUSUM on T²
k_c = 0.5 * sigma_T2
h_c = 5.0 * sigma_T2
cusum_p = np.zeros(len(T2_all))
cusum_alerts = np.zeros(len(T2_all), dtype=bool)
for i in range(1, len(T2_all)):
    cusum_p[i] = max(0, cusum_p[i-1] + (T2_all[i] - mu_T2) - k_c)
    cusum_alerts[i] = cusum_p[i] > h_c

# Joint alert: EWMA OR CUSUM
joint_alerts = ewma_alerts | cusum_alerts

print(f'Detection counts:')
print(f'  T² fixed UCL:  {violations_T2.sum():5d} alarms')
print(f'  EWMA:          {ewma_alerts.sum():5d} alarms')
print(f'  CUSUM:         {cusum_alerts.sum():5d} alarms')
print(f'  Joint (OR):    {joint_alerts.sum():5d} alarms')

In [ ]:
# ─── 7. Detection latency analysis ───────────────────────
# For each fault, compute how many samples after fault onset the first alert fires
fault_ranges = {1: (500, 700), 2: (1100, 1300),
                3: (1800, 2000), 4: (2300, 2500)}
methods = {'T² UCL': violations_T2, 'EWMA': ewma_alerts,
           'CUSUM': cusum_alerts, 'Joint': joint_alerts}

latency_records = []
for fid, (start, end) in fault_ranges.items():
    for mname, alerts in methods.items():
        # First alert at or after fault start
        alerts_in_fault = np.where(alerts[start:end])[0]
        if len(alerts_in_fault) > 0:
            latency = alerts_in_fault[0]
        else:
            latency = end - start   # missed entirely
        latency_records.append(
            {'Fault': fid, 'Method': mname, 'Latency (samples)': latency})

latency_df = pd.DataFrame(latency_records).pivot(
    index='Fault', columns='Method', values='Latency (samples)')

fig, ax = plt.subplots(figsize=(10, 5))
sns.heatmap(latency_df, annot=True, fmt='.0f', cmap='RdYlGn_r', ax=ax,
            cbar_kws={'label': 'Samples to first alert'}, linewidths=0.5)
ax.set_title('Detection Latency Heatmap (lower = faster)', fontsize=13)
ax.set_xlabel('Detection Method')
ax.set_ylabel('Fault ID')
plt.tight_layout()
plt.show()

print('\nLatency Table:')
print(latency_df.to_string())

5. Structured Residual Analysis (SRA) — Fault Diagnosis


In [ ]:
# ─── 8. SRA: isolate faults by residual structure ──────────
# For each variable, build a regression model from the others (Phase I).
# The residual = actual - predicted captures variable-specific deviations.

from sklearn.linear_model import Ridge

phase1_data = df[VAR_NAMES].iloc[:PHASE1_END].values
sra_models = {}
for i, var in enumerate(VAR_NAMES):
    X_others = np.delete(phase1_data, i, axis=1)
    y_target = phase1_data[:, i]
    model = Ridge(alpha=1.0)
    model.fit(X_others, y_target)
    sra_models[var] = model

# Compute residuals for all samples
all_data = df[VAR_NAMES].values
residuals = np.zeros_like(all_data)
for i, var in enumerate(VAR_NAMES):
    X_others = np.delete(all_data, i, axis=1)
    residuals[:, i] = all_data[:, i] - sra_models[var].predict(X_others)

# Normalise by Phase I residual std
phase1_resid_std = np.std(residuals[:PHASE1_END], axis=0) + 1e-10
residuals_norm = residuals / phase1_resid_std

print('Residual matrix shape:', residuals_norm.shape)

In [ ]:
# ─── 9. SRA heatmap + per-fault diagnosis ─────────────────
fig, axes = plt.subplots(2, 1, figsize=(16, 9), sharex=True)

# Residual heatmap (subsample for visibility)
step = 3
sns.heatmap(residuals_norm[::step].T,
            ax=axes[0], cmap='coolwarm', vmin=-3, vmax=3,
            xticklabels=False,
            yticklabels=VAR_NAMES,
            cbar_kws={'label': 'Normalised Residual'})
axes[0].set_title('Structured Residual Analysis — Heatmap', fontsize=13)
axes[0].set_ylabel('')

# Mark fault boundaries
for fid, (start, end) in fault_ranges.items():
    axes[0].axvline(start // step, color='black', ls='-', lw=1.5, alpha=0.7)
    axes[0].axvline(end // step,   color='black', ls='--', lw=1.0, alpha=0.5)
    axes[0].text((start+end)//(2*step), 6.2,
                 f'F{fid}', fontsize=9, ha='center', fontweight='bold')

# Per-fault mean |residual| (diagnosis signature)
axes[1].set_visible(False)

fig2, ax2 = plt.subplots(figsize=(12, 5))
for fid, (start, end) in fault_ranges.items():
    mean_abs_resid = np.abs(residuals_norm[start:end]).mean(axis=0)
    ax2.plot(VAR_NAMES, mean_abs_resid, 'o-',
             lw=1.5, ms=6, label=f'Fault {fid}')

ax2.set_title(
    'Mean |Residual| per Variable — Fault Diagnosis Signatures', fontsize=13)
ax2.set_ylabel('Mean |Normalised Residual|')
ax2.set_xlabel('Process Variable')
ax2.legend()
ax2.tick_params(axis='x', rotation=15)
plt.tight_layout()
plt.show()

In [ ]:
# ─── 10. Diagnosis summary table ──────────────────────────
print('\n' + '═'*60)
print(' FAULT DIAGNOSIS SUMMARY')
print('═'*60)
for fid, (start, end) in fault_ranges.items():
    mean_resid = np.abs(residuals_norm[start:end]).mean(axis=0)
    ranked = np.argsort(mean_resid)[::-1]
    print(f'\n  Fault {fid}: ({start}–{end})')
    print(f'    Root-cause ranking:')
    for rank, idx in enumerate(ranked[:3]):
        print(
            f'      {rank+1}. {VAR_NAMES[idx]:15s} (mean |resid| = {mean_resid[idx]:.3f})')

---
## Summary & Portfolio Takeaways

| Technique | Value |
|---|---|
| **Phase I / Phase II** | Proper two-phase methodology — baseline established on known-normal data only |
| **Hotelling T²** | Single multivariate statistic captures correlated process shifts that univariate charts miss |
| **SPE Chart** | Catches model-structure violations (Fault 3: variance, Fault 4: correlation breakdown) |
| **PCA Contribution** | Decomposes T² alarm into per-variable contributions — instant root-cause identification |
| **EWMA + CUSUM Joint** | Detects subtle drifts (Fault 1) faster than fixed UCL; joint detection minimises latency |
| **SRA** | Regression-based residual isolation — produces ranked diagnosis signatures per fault type |
| **Latency Analysis** | Quantitative comparison shows Joint detection is fastest across all fault types |

This is a complete **multivariate statistical process control** system — ready for **chemical plants**, **semiconductor fabs**, and **pharmaceutical manufacturing**.
